In [1]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.2 MB/s eta 0:00:00


In [2]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
pd.options.display.max_columns = 500

connection = psycopg2.connect(
    database="startml",
    user="robot-startml-ro",
    password="pheiph0hahj1Vaif",
    host="postgres.lab.karpov.courses",
    port=6432
)

cursor = connection.cursor()
cursor.execute("""SELECT * FROM public.user_data""")
result = cursor.fetchall()

df_user = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

cursor.execute("""SELECT * FROM public.post_text_df""")
result = cursor.fetchall()

df_post = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

cursor.execute("""SELECT * FROM public.feed_data limit 300000""")
result = cursor.fetchall()

df_feed = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
cursor.close()
connection.close()
df_feed = df_feed.sort_values("timestamp")
df_post

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [3]:
df_user

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [4]:
df_feed

,timestamp,user_id,post_id,action,target
143769,2021-10-01 06:08:37,80377,5335,view,0
143770,2021-10-01 06:10:18,80377,6764,view,0
143771,2021-10-01 06:13:14,80377,6287,view,0
143772,2021-10-01 06:13:53,80377,291,view,0
143773,2021-10-01 06:14:23,80377,2461,view,0
...,...,...,...,...,...
2818,2021-12-29 23:37:52,163679,4259,view,0
103770,2021-12-29 23:37:52,101364,703,view,0
2819,2021-12-29 23:39:10,163679,7002,view,1
103771,2021-12-29 23:39:10,101364,1870,view,0


In [5]:
post_text = df_post[['post_id','text']]
post_text.set_index('post_id', inplace=True)
post_text

,text
post_id,
1,UK economy facing major risks\n\nThe UK manufa...
2,Aids and climate top Davos agenda\n\nClimate c...
3,Asian quake hits European shares\n\nShares in ...
4,India power shares jump on debut\n\nShares in ...
5,Lacroix label bought by US firm\n\nLuxury good...
...,...
7315,"OK, I would not normally watch a Farrelly brot..."
7316,I give this movie 2 stars purely because of it...
7317,I cant believe this film was allowed to be mad...


In [6]:
import numpy as np
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from tqdm import tqdm

dataset = post_text

from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import \
    RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import \
    DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel


def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(
        checkpoint_names[model_name])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(torch.cuda.get_device_name())
tokenizer, model = get_model('roberta')
model = model.to(device)

# Функция для получения эмбеддингов из текста
def get_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: inputs[key].to(device) for key in inputs}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

# Создание эмбеддингов для всех строк в датафрейме
embeddings_list = []
for index, row in post_text.iterrows():
    text = row['text']
    embeddings = get_embeddings(text)
    embeddings = embeddings.cpu().numpy()  # Копирование тензора на CPU и преобразование в массив NumPy
    embeddings_list.append(embeddings)


print(len(embeddings_list))

2024-06-21 08:48:21.517093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 08:48:21.517194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 08:48:21.644419: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Tesla P100-PCIE-16GB


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


7023


In [8]:
# Объединение датафреймов embeddings_df и df_post[['post_id', 'topic']]
import pandas as pd
from catboost import CatBoostClassifier

import numpy as np

# Преобразование трехмерного массива эмбеддингов в двумерный массив
X = np.array(embeddings_list)
X = X.reshape(X.shape[0], -1)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Стандартизация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Применение PCA для уменьшения размерности
pca = PCA(n_components=10)  # Указываем количество компонент, на которые хотим уменьшить размерность
X_pca = pca.fit_transform(X_scaled)

# Создание датафрейма с уменьшенной размерностью
data_pca = pd.concat([df_post[['post_id', 'topic']], pd.DataFrame(X_pca)], axis=1)

data_pca

,post_id,topic,0,1,2,3,4,5,6,7,8,9
0,1,business,-3.514155,15.753833,-0.298763,-3.163678,-5.370112,-1.589024,1.796534,-1.731818,-6.138156,6.192593
1,2,business,-2.159950,13.495478,-3.992386,0.469858,-2.693444,1.249408,0.865693,0.009316,-1.465058,2.278529
2,3,business,-6.839348,13.130909,1.943464,-3.410149,-4.546646,-0.365237,7.870173,8.167789,-4.031091,4.784634
3,4,business,-0.310258,9.237336,-2.629246,-1.670174,-8.316555,-1.659710,4.319267,6.199043,-5.792403,4.314416
4,5,business,-1.366239,8.929032,-0.746582,1.024194,-5.781115,-1.127661,3.282559,2.978265,-5.908419,2.953598
...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,movie,-8.186323,-6.897585,-2.899936,-2.427538,3.024574,-4.253700,0.764583,-3.282684,-0.311872,2.398428
7019,7316,movie,-11.632481,-10.653990,-1.360517,-5.156749,-2.418425,-1.018914,-2.159425,3.995406,-2.001221,2.681316
7020,7317,movie,-6.104027,-10.072062,-2.866875,-6.967229,0.808511,-0.599596,0.534537,2.358672,2.589679,4.580222
7021,7318,movie,-13.204796,-7.380834,3.753697,2.823107,-2.737210,0.133603,-0.868949,-1.187026,-1.733192,-0.959571


In [9]:
from catboost import CatBoostClassifier
catboost_model = CatBoostClassifier(learning_rate=0.02)

data = pd.merge(df_feed, df_user, left_on='user_id', right_on='user_id', how='left')
# Преобразуем столбец timestamp в формат datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
# Выделяем год в отдельный столбец
data['year'] = data['timestamp'].dt.year
# Выделяем номер месяца в отдельный столбец
data['month'] = data['timestamp'].dt.month
# Выводим результат
data = data.drop('timestamp', axis=1)
data.drop('action', axis=1)
data

,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,year,month
0,80377,5335,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10
1,80377,6764,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10
2,80377,6287,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10
3,80377,291,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10
4,80377,2461,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,163679,4259,view,0,1,20,Russia,Moscow,4,Android,organic,2021,12
299996,101364,703,view,0,0,44,Russia,Moscow,0,iOS,ads,2021,12
299997,163679,7002,view,1,1,20,Russia,Moscow,4,Android,organic,2021,12
299998,101364,1870,view,0,0,44,Russia,Moscow,0,iOS,ads,2021,12


In [10]:
data = pd.merge(data, data_pca, left_on='post_id', right_on='post_id', how='left')
data

,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,year,month,topic,0,1,2,3,4,5,6,7,8,9
0,80377,5335,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-9.626642,-10.179944,-3.611641,-4.458071,-1.883648,-6.203027,0.720282,-1.268615,-0.983945,-0.692143
1,80377,6764,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-13.115120,-5.247258,1.741011,-1.060290,4.150322,5.544623,3.116892,1.375109,3.972821,0.337208
2,80377,6287,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-14.228318,-11.091353,8.841573,13.925123,-6.621311,-1.548708,-4.226734,2.977926,-3.900430,1.506373
3,80377,291,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10,business,-2.871715,9.321619,0.173737,1.189121,-4.224099,0.476426,3.580690,5.124219,-5.576206,0.587674
4,80377,2461,view,0,1,43,Russia,Tolyatti,4,Android,ads,2021,10,covid,25.037582,-0.369002,-2.735413,2.254350,-1.379193,-1.427846,0.108274,1.832054,1.631565,2.330455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,163679,4259,view,0,1,20,Russia,Moscow,4,Android,organic,2021,12,movie,-14.259285,-6.611391,-1.154022,2.244344,-0.994426,4.745672,1.694730,-0.197383,0.084673,4.875067
299996,101364,703,view,0,0,44,Russia,Moscow,0,iOS,ads,2021,12,entertainment,-1.336519,8.753925,-3.882206,5.710359,-2.934283,-6.281216,1.319166,-4.647166,3.402884,-1.797388
299997,163679,7002,view,1,1,20,Russia,Moscow,4,Android,organic,2021,12,movie,-9.247227,-3.744427,0.517755,-1.456132,-6.647545,1.190310,0.309141,-2.045822,-0.614695,-5.709553
299998,101364,1870,view,0,0,44,Russia,Moscow,0,iOS,ads,2021,12,sport,1.632280,6.468411,0.787476,4.266459,2.628868,-10.789705,6.446001,-2.603127,5.792636,3.681168


In [11]:
X_train = data.iloc[:-80000].copy().drop(['target', 'action'], axis=1)
X_test = data.iloc[-80000:].copy().drop(['target', 'action'], axis=1)
y_train = data['target'].iloc[:-80000].copy()
y_test = data['target'].iloc[-80000:].copy()
X_train

,user_id,post_id,gender,age,country,city,exp_group,os,source,year,month,topic,0,1,2,3,4,5,6,7,8,9
0,80377,5335,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-9.626642,-10.179944,-3.611641,-4.458071,-1.883648,-6.203027,0.720282,-1.268615,-0.983945,-0.692143
1,80377,6764,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-13.115120,-5.247258,1.741011,-1.060290,4.150322,5.544623,3.116892,1.375109,3.972821,0.337208
2,80377,6287,1,43,Russia,Tolyatti,4,Android,ads,2021,10,movie,-14.228318,-11.091353,8.841573,13.925123,-6.621311,-1.548708,-4.226734,2.977926,-3.900430,1.506373
3,80377,291,1,43,Russia,Tolyatti,4,Android,ads,2021,10,business,-2.871715,9.321619,0.173737,1.189121,-4.224099,0.476426,3.580690,5.124219,-5.576206,0.587674
4,80377,2461,1,43,Russia,Tolyatti,4,Android,ads,2021,10,covid,25.037582,-0.369002,-2.735413,2.254350,-1.379193,-1.427846,0.108274,1.832054,1.631565,2.330455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219995,22534,6482,0,23,Russia,Saint Petersburg,4,iOS,ads,2021,12,movie,-8.499927,-9.625461,-2.275497,-3.745173,0.208615,-0.642049,-3.348671,3.319548,0.312443,0.634644
219996,60042,710,1,23,Russia,Moscow,1,Android,ads,2021,12,entertainment,-5.116072,5.593825,-0.644673,2.467247,-10.042808,-6.826029,0.602789,0.441321,-7.458834,0.255905
219997,163738,6449,0,27,Russia,Moscow,2,iOS,organic,2021,12,movie,-5.346433,-3.402427,-1.758391,4.469805,-1.904497,-1.068510,-0.377712,0.383578,0.668578,-2.082416
219998,101426,461,1,35,Russia,Moscow,3,Android,ads,2021,12,business,-0.647474,14.753071,-4.570552,-1.921973,-4.115400,1.961484,2.705575,-2.017506,0.574638,1.119635


In [12]:
cat_features = ['age', 'country', 'city', 'os', 'source','topic']
catboost_model.fit(X_train, y_train, cat_features)
from sklearn.metrics import accuracy_score
y_train_pred = catboost_model.predict(X_train)
y_test_pred = catboost_model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy on train set: {train_accuracy}')
print(f'Accuracy on test set: {test_accuracy}')
y_test_array = y_test.to_numpy()
probs = catboost_model.predict_proba(X_test)
# Получаем индексы вероятностей, отсортированные по убыванию
sorted_indexes = np.argsort(probs, axis=1)[:, ::-1]

# Проверяем, содержат ли истинные метки в топ-5 предсказанных классов
hits = []
for i in range(len(y_test_array)):
    true_label = y_test_array[i]
    top5_labels = sorted_indexes[i, :5] + 1  # Добавляем 1 к индексам
    if true_label in top5_labels:
        hits.append(1)
    else:
        hits.append(0)

# Вычисляем HitRate@5
hit_rate_at_5 = np.mean(hits)
print("HitRate@5:", hit_rate_at_5)

0:	learn: 0.6759942	total: 236ms	remaining: 3m 56s
1:	learn: 0.6596632	total: 413ms	remaining: 3m 25s
2:	learn: 0.6440503	total: 572ms	remaining: 3m 9s
3:	learn: 0.6291432	total: 707ms	remaining: 2m 56s
4:	learn: 0.6149684	total: 867ms	remaining: 2m 52s
5:	learn: 0.6015987	total: 1.02s	remaining: 2m 48s
6:	learn: 0.5887066	total: 1.18s	remaining: 2m 47s
7:	learn: 0.5765774	total: 1.33s	remaining: 2m 45s
8:	learn: 0.5650251	total: 1.48s	remaining: 2m 42s
9:	learn: 0.5539574	total: 1.62s	remaining: 2m 40s
10:	learn: 0.5434112	total: 1.77s	remaining: 2m 39s
11:	learn: 0.5334522	total: 1.86s	remaining: 2m 33s
12:	learn: 0.5239547	total: 2.02s	remaining: 2m 33s
13:	learn: 0.5147476	total: 2.17s	remaining: 2m 33s
14:	learn: 0.5061565	total: 2.24s	remaining: 2m 27s
15:	learn: 0.4978769	total: 2.33s	remaining: 2m 23s
16:	learn: 0.4900264	total: 2.46s	remaining: 2m 22s
17:	learn: 0.4825218	total: 2.53s	remaining: 2m 17s
18:	learn: 0.4754585	total: 2.6s	remaining: 2m 13s
19:	learn: 0.4687381	tot

In [13]:
import os
from catboost import CatBoostClassifier


def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("/my/super/path")
    from_file = CatBoostClassifier()

    return from_file.load_model(model_path)

catboost_model.save_model('catboost_model',
                           format="cbm")

from_file = CatBoostClassifier()  # здесь не указываем параметры, которые были при обучении, в дампе модели все есть

from_file.load_model("catboost_model")

from_file.predict(X_train)

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
pd.options.display.max_columns = 500

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml********"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)
df_feed_all_user = batch_load_sql('''WITH ranked_posts AS (
    SELECT user_id, post_id, timestamp, target,
           ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY target DESC, post_id) AS post_rank
    FROM feed_data)
SELECT user_id, post_id, timestamp, target
FROM ranked_posts
WHERE post_rank <= 5''')

data = pd.merge(df_feed_all_user, df_user, left_on='user_id', right_on='user_id', how='left')
# Преобразуем столбец timestamp в формат datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Выделяем год в отдельный столбец
data['year'] = data['timestamp'].dt.year

# Выделяем номер месяца в отдельный столбец
data['month'] = data['timestamp'].dt.month

# Выводим результат
data = data.drop('timestamp', axis=1)
data = data.drop('target', axis=1)

data = pd.merge(data, data_pca, left_on='post_id', right_on='post_id', how='left')
print(data)

        user_id  post_id  gender  age country         city  exp_group  \
0           200      249       1   34  Russia    Degtyarsk          3   
1           200      472       1   34  Russia    Degtyarsk          3   
2           200      668       1   34  Russia    Degtyarsk          3   
3           200      783       1   34  Russia    Degtyarsk          3   
4           200      805       1   34  Russia    Degtyarsk          3   
...         ...      ...     ...  ...     ...          ...        ...   
816020   168552     1662       1   16  Russia  Ivanteyevka          4   
816021   168552     1693       1   16  Russia  Ivanteyevka          4   
816022   168552     1810       1   16  Russia  Ivanteyevka          4   
816023   168552     1823       1   16  Russia  Ivanteyevka          4   
816024   168552     1915       1   16  Russia  Ivanteyevka          4   

             os   source  year  month          topic         0          1  \
0       Android      ads  2021     11       bu

In [15]:
import pandas as pd
from sqlalchemy import create_engine
from catboost import CatBoostClassifier

# Создание соединения с базой данных
engine = create_engine(
    "postgresql://robot-startml*********"
    "postgres.lab.karpov.courses:6432/startml"
)

# Сохранение признаков в таблицу
def save_features(features: pd.DataFrame):
    features.to_sql(
        name='*******lesson_10',
        con=engine,
        if_exists='replace',
        index=False
    )

# Загрузка признаков из таблицы
def load_features() -> pd.DataFrame:
    query = "SELECT * FROM *******lesson_10"
    return batch_load_sql(query)


save_features(data)